MOVIE RECOMMENDATION SYSTEM

IMPORT LIBRARIES

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

READ FILES: USER RATINGS ON MOVIES

In [9]:
column_names = ['user_id','movie_id','rating']
ratings = pd.read_csv('u.data', sep="\t", names = column_names, usecols=range(3), encoding="ISO-8859-1")
movie_titles = ['movie_id','title']
movies = pd.read_csv('u.item', sep='|', names = movie_titles, usecols = range(2), encoding = "ISO-8859-1")

FileNotFoundError: [Errno 2] No such file or directory: 'u.data'

In [49]:
ratings.head()

user_id  movie_id  rating
0      196       242       3
1      186       302       3
2       22       377       1
3      244        51       2
4      166       346       1

In [50]:
movies.head()

movie_id              title
0         1   Toy Story (1995)
1         2   GoldenEye (1995)
2         3  Four Rooms (1995)
3         4  Get Shorty (1995)
4         5     Copycat (1995)

MERGE BOTH USER AND MOVIE DATA

In [41]:
ratings_df = pd.merge(movies,ratings)
ratings_df.head()

movie_id             title  user_id  rating
0         1  Toy Story (1995)      308       4
1         1  Toy Story (1995)      287       5
2         1  Toy Story (1995)      148       4
3         1  Toy Story (1995)      280       4
4         1  Toy Story (1995)       66       3

HIGHEST RATED MOVIES

In [54]:
ratings_df.groupby('title')['rating'].count().sort_values(ascending=False).head()

title
Star Wars (1977)             583
Contact (1997)               509
Fargo (1996)                 508
Return of the Jedi (1983)    507
Liar Liar (1997)             485
Name: rating, dtype: int64

PIVOT TABLE: VIEW WHAT USERS HAD WATCHED AND RATED

In [30]:
movie_ratings = ratings_df.pivot_table(index='user_id',columns ='title',values = 'rating')
movie_ratings.head()

title    'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  \
user_id                                                                   
1                              NaN           NaN                    2.0   
2                              NaN           NaN                    NaN   
3                              NaN           NaN                    NaN   
4                              NaN           NaN                    NaN   
5                              NaN           NaN                    2.0   

title    12 Angry Men (1957)  187 (1997)  2 Days in the Valley (1996)  \
user_id                                                                 
1                        5.0         NaN                          NaN   
2                        NaN         NaN                          NaN   
3                        NaN         2.0                          NaN   
4                        NaN         NaN                          NaN   
5                        NaN         NaN                          NaN   

title    20,000 Leagues Under the Sea (1954)  2001: A Space Odyssey (1968)  \
user_id                                                                      
1                                        3.0                           4.0   
2                                        NaN                           NaN   
3                                        NaN                           NaN   
4                                        NaN                           NaN   
5                                        NaN                           4.0   

title    3 Ninjas: High Noon At Mega Mountain (1998)  39 Steps, The (1935)  \
user_id                                                                      
1                                                NaN                   NaN   
2                                                1.0                   NaN   
3                                                NaN                   NaN   
4                                                NaN                   NaN   
5                                                NaN                   NaN   

title    ...  Yankee Zulu (1994)  Year of the Horse (1997)  \
user_id  ...                                                 
1        ...                 NaN                       NaN   
2        ...                 NaN                       NaN   
3        ...                 NaN                       NaN   
4        ...                 NaN                       NaN   
5        ...                 NaN                       NaN   

title    You So Crazy (1994)  Young Frankenstein (1974)  Young Guns (1988)  \
user_id                                                                      
1                        NaN                        5.0                3.0   
2                        NaN                        NaN                NaN   
3                        NaN                        NaN                NaN   
4                        NaN                        NaN                NaN   
5                        NaN                        4.0                NaN   

title    Young Guns II (1990)  Young Poisoner's Handbook, The (1995)  \
user_id                                                                
1                         NaN                                    NaN   
2                         NaN                                    NaN   
3                         NaN                                    NaN   
4                         NaN                                    NaN   
5                         NaN                                    NaN   

title    Zeus and Roxanne (1997)  unknown  Á köldum klaka (Cold Fever) (1994)  
user_id                                                                        
1                            NaN      4.0                                 NaN  
2                            NaN      NaN                                 NaN  
3                            NaN      NaN                    

TAKE THE HIGHEST RATED MOVIE - STAR WARS

In [56]:
starwars_user_ratings = movie_ratings['Star Wars (1977)']
starwars_user_ratings.head()

user_id
1    5.0
2    5.0
3    NaN
4    5.0
5    4.0
Name: Star Wars (1977), dtype: float64

FIND CORRELATION OF STARWAR MOVIE AND RECOMMENDED MOVIES RELATED BASED ON HIGH RATINGS AND DROP RESULTS THAT DON'T HAVE DATA

In [58]:
similar_to_starwars = movie_ratings.corrwith(starwars_user_ratings)
corr_starwars = pd.DataFrame(similar_to_starwars,columns=['correlation'])
corr_starwars.dropna(inplace=True)

In [59]:
corr_starwars.sort_values('correlation', ascending=False).head(10)

correlation
title                                                          
Hollow Reed (1996)                                          1.0
Commandments (1997)                                         1.0
Cosi (1996)                                                 1.0
No Escape (1994)                                            1.0
Stripes (1981)                                              1.0
Star Wars (1977)                                            1.0
Man of the Year (1995)                                      1.0
Beans of Egypt, Maine, The (1994)                           1.0
Old Lady Who Walked in the Sea, The (Vieille qu...          1.0
Outlaw, The (1943)                                          1.0

NUMBER OF RATES AND AVERAGE

In [61]:
movie_stats = ratings_df.groupby('title').agg({'rating': [np.size, np.mean]})
movie_stats.head()

rating          
                            size      mean
title                                     
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

POPULAR MOVIES: OVER 100 NUMBER OF AUDIENCES

In [63]:
popular_movies = movie_stats['rating']['size']>=100
movie_stats[popular_movies].sort_values([('rating','mean')], ascending=False)[:10]

rating          
                                   size      mean
title                                            
Close Shave, A (1995)               112  4.491071
Schindler's List (1993)             298  4.466443
Wrong Trousers, The (1993)          118  4.466102
Casablanca (1942)                   243  4.456790
Shawshank Redemption, The (1994)    283  4.445230
Rear Window (1954)                  209  4.387560
Usual Suspects, The (1995)          267  4.385768
Star Wars (1977)                    583  4.358491
12 Angry Men (1957)                 125  4.344000
Citizen Kane (1941)                 198  4.292929

JOINING POPULAR MOVIES DATA SET WITH STARWARS TO COMPARE

In [66]:
popular_df = movie_stats[popular_movies].join(pd.DataFrame(similar_to_starwars, columns = ['similarity']))
popular_df.head

<bound method NDFrame.head of                                               (rating, size)  (rating, mean)  \
title                                                                          
101 Dalmatians (1996)                                    109        2.908257   
12 Angry Men (1957)                                      125        4.344000   
2001: A Space Odyssey (1968)                             259        3.969112   
Absolute Power (1997)                                    127        3.370079   
Abyss, The (1989)                                        151        3.589404   
...                                                      ...             ...   
Willy Wonka and the Chocolate Factory (1971)             326        3.631902   
Wizard of Oz, The (1939)                                 246        4.077236   
Wrong Trousers, The (1993)                               118        4.466102   
Young Frankenstein (1974)                                200        3.945000   
Young Guns

HIGHEST SIMILARITY

In [69]:
popular_df.sort_values(['similarity'],ascending=False)[:10]

(rating, size)  \
title                                                                
Star Wars (1977)                                               583   
Empire Strikes Back, The (1980)                                367   
Return of the Jedi (1983)                                      507   
Raiders of the Lost Ark (1981)                                 420   
Austin Powers: International Man of Mystery (1997)             130   
Sting, The (1973)                                              241   
Indiana Jones and the Last Crusade (1989)                      331   
Pinocchio (1940)                                               101   
Frighteners, The (1996)                                        115   
L.A. Confidential (1997)                                       297   

                                                    (rating, mean)  similarity  
title                                                                           
Star Wars (1977)                                          4.358491    1.000000  
Empire Strikes Back, The (1980)                           4.204360    0.747981  
Return of the Jedi (1983)                                 4.007890    0.672556  
Raiders of the Lost Ark (1981)                            4.252381    0.536117  
Austin Powers: International Man of Mystery (1997)        3.246154    0.377433  
Sting, The (1973)                                         4.058091    0.367538  
Indiana Jones and the Last Crusade (1989)                 3.930514    0.350107  
Pinocchio (1940)                                          3.673267    0.347868  
Frighteners, The (1996)                                   3.234783    0.332729  
L.A. Confidential (1997)                                  4.161616    0.319065